#### importing packages

In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from irc6_2007 import *

from reaction import find_bm, find_sf, find_ra, find_rb
# from bridge_specs import box, bearing

bridge specs

In [2]:
span = 50
c_c = 1.5

# live load calculations

#### defining load (vehicles)
load pair contains pair (wheel position, wheel load)

In [3]:
vehicles = [ll_A, ll_70R, ll_70RT]
classA_pair, class70R, class70RT = [list(i.loadpair) for i in vehicles]
loads = [classA_pair, class70R, class70RT]

#### maxBM, maxSF at equal intervals
when each vehicle from the list of vehicles travels along the span, max reactions (BM and SF+/-) at equally divided intervals are calculated and stored in maxBMs, maxSFs_plus, maxSFs_minus.

In [4]:


maxBMs = []
maxSFs_plus = []
maxSFs_minus = []
for i in range(len(loads)):
    maxBM = []
    maxSF_plus = []
    maxSF_minus = []
    for j in range(9):
        at = span / 8 * j
        first_wheel_at = 0
        step = 0.1

        BM = find_bm(span, 0, at)
        SF_plus = find_sf(span, 0, at)
        SF_minus = find_sf(span, 0, at)
        for k in range(int((span + loads[i][-1][0]) / step) + 2):  # '+2' to make sure the load moves all the way to
            # end until it has no effect
            bm = 0
            sf = 0
            for this in loads[i]:
                a, load = this
                pos = -a + first_wheel_at
                bm = bm + find_bm(span, pos, at) * load
                sf = sf + find_sf(span, pos, at) * load
            first_wheel_at += step
            BM = bm if bm > BM else BM
            SF_plus = sf if sf > SF_plus else SF_plus
            SF_minus = sf if sf < SF_minus else SF_minus
        maxBM.append(round(BM, 3))
        maxSF_plus.append(round(SF_plus, 3))
        maxSF_minus.append(round(SF_minus, 3))
    maxBMs.append(maxBM)
    maxSFs_plus.append(maxSF_plus)
    maxSFs_minus.append(maxSF_minus)

#### make a dataframe that contains maxBMs, maxSFs_plus, maxSFs_minus

In [5]:
A = ['MaxBM', 'MaxSF+', 'MaxSF-']
B = ['ClassA', 'Class70RW', 'Class70RT']

# A = ['ClassA', 'Class70RW', 'Class70RT']
# B = ['MaxBM', 'MaxSF+', 'MaxSF-']

iterables = [A, B]
index = pd.MultiIndex.from_product(iterables)

C = []

for i in [maxBMs, maxSFs_plus, maxSFs_minus]:
    C.extend(i)

# for i in range(len(loads)):
#     for j in [maxBMs, maxSFs_plus, maxSFs_minus]:
#         C.append(j[i])

df = pd.DataFrame(C, index=index, columns=[span / 8 * i for i in range(9)])
# print(df.loc[('ClassA', 'MaxSF-')])   ## you can navigate using loc, iloc

new_row = df.loc['MaxSF+'].where(df.loc['MaxSF+'] > abs(df.loc['MaxSF-']), abs(df.loc['MaxSF-']))

new_row.index = pd.MultiIndex.from_product([['MaxSF'], B])
df = pd.concat([df, new_row])

# df.to_excel('outputs/loads.xlsx') # uncomment to save as excel file

In [6]:
df

0.00      6.25      12.50      18.75      25.00  \
MaxBM  ClassA       0.000  2361.037  4068.075   5120.737   5526.150   
       Class70RW    0.000  4826.025  8200.950  10182.475  10877.400   
       Class70RT    0.000  3651.438  6259.675   7823.212   8342.050   
MaxSF+ ClassA     445.318   376.622   307.926    238.122    169.426   
       Class70RW  895.524   771.524   647.524    521.524    397.524   
       Class70RT  667.590   579.390   492.590    404.390    317.590   
MaxSF- ClassA       0.000   -12.030   -51.234   -106.602   -175.162   
       Class70RW    0.000   -17.024   -87.536   -208.476   -332.476   
       Class70RT    0.000   -54.810  -143.010   -229.810   -316.610   
MaxSF  ClassA     445.318   376.622   307.926    238.122    175.162   
       Class70RW  895.524   771.524   647.524    521.524    397.524   
       Class70RT  667.590   579.390   492.590    404.390    317.590   

                      31.25     37.50     43.75    50.00  
MaxBM  ClassA      5340.038  4327.675  2494.638    0.000  
       Class70RW  10163.225  8053.750  4611.975    0.000  
       Class70RT   7823.588  6260.425  3652.563    0.000  
MaxSF+ ClassA        99.676    43.520    13.056   -0.000  
       Class70RW    271.524   148.964    44.948   -0.000  
       Class70RT    229.390   142.590    54.390   -0.000  
MaxSF- ClassA      -244.966  -313.662  -383.466 -452.162  
       Class70RW   -458.476  -582.476  -708.476 -832.476  
       Class70RT   -404.810  -491.610  -579.810 -666.610  
MaxSF  ClassA       244.966   313.662   383.466  452.162  
       Class70RW    458.476   582.476   708.476  832.476  
       Class70RT    404.810   491.610   579.810  666.610

*how to read Excel file*

In [7]:
# df = pd.read_excel('outputs/loads.xlsx', index_col=[0, 1])

# ###get index names###
# A = df.index.get_level_values(0).drop_duplicates().to_list()
# B = df.index.get_level_values(1).drop_duplicates().to_list()
# print(A, B)

#### impact factor

In [8]:
IF = [impact(i.name, span) for i in vehicles]

[0.088, 0.088, 0.088]

#### possible combinations for given carriageway width

In [9]:
cwWidth = 6
carriageway = Carriageway(cwWidth)
combinations = carriageway.combinations()
combination_val = [i.get_value() for i in combinations]

#### eccentricity calculation
a combination consists of multiple arrangements gives different eccentricities. maximum eccentricities for each combination calculated.

In [10]:
cw = 6

carriageway = Carriageway(width=cw)
combinations = carriageway.combinations()
list_combinations = []
list_max_e = []

for i in combinations:
    list_combinations.append([i.classA, i.class70Rw, i.class70Rt])
    list_max_e.append(i.max_e())


#### make a dataframe for combinations and their corresponding eccentricities

In [11]:
df1 = pd.DataFrame(list_combinations, columns=['ClassA', 'Class70Rw', 'Class70Rt'],
                   index=[f'comb{i + 1}' for i in range(len(list_combinations))])
df1['MaxEccentricity'] = list_max_e

*export eccentricity as excel file (uncomment)*

In [12]:
# df1.to_excel('outputs/max_e.xlsx')

#### total reactions for given combination
total reactions = sum of (maxreactions at mid span * impact factor * no. of that vehicles)

In [48]:
totalBMs = [(df.loc['MaxBM'][span / 2] + df.loc['MaxBM'][span / 2] * IF).dot(combination_val[i]) for i in range(len(combinations))]
totalSFs = [(df.loc['MaxSF'][span / 2] + df.loc['MaxSF'][span / 2] * IF).dot(combination_val[i]) for i in range(len(combinations))]

In [50]:
df1['TotalBM'] = totalBMs
df1['TotalSF'] = totalSFs

In [51]:
df1

,ClassA,Class70Rw,Class70Rt,MaxEccentricity,TotalBM,TotalSF
comb1,2,0,0,-3.739699e-16,12024.9024,381.152512
comb2,0,1,0,-4.050000e-01,11834.6112,432.506112
comb3,0,0,1,-3.500000e-01,9076.1504,345.537920


# loads on peir

pier specs

In [52]:
pier_width = 1.6
pier_height_hfl = 7 # height of HFL from pier base

**live load components**

In [53]:
dfSF = pd.DataFrame()    # accessing df takes longer
dfBM = pd.DataFrame()
        #[a, 70r, 70rt]
veh_no = [2, 2, 1] # revise this !!!
for index, veh in enumerate(vehicles):
    new_veh = veh
    for i in range(veh_no[index]-1):
        new_veh+=veh
    final_load = list(new_veh.loadpair)

    max_sum_at = 0
    sumlr = 0
    difflr = 0
    nleft = 0
    nright = 0

    head_at = 0
    step = 0.01
    for i in range(int((span * 2 + final_load[-1][0]) / step)):
        left = 0
        right = 0
        for j in final_load:
            rb = find_rb(span, head_at - j[0], c_c / 2, c_c / 2) * j[1]
            ra = find_ra(span, head_at - j[0] - (span + c_c), c_c / 2, c_c / 2) * j[1]
            temp_load = 0
            if head_at - j[0] == span + c_c / 2:
                temp_load = (j[1]) / 2
                ra = find_ra(span, head_at - j[0] - (span + c_c), c_c / 2, c_c / 2) * temp_load
                rb = find_rb(span, head_at - j[0], c_c / 2, c_c / 2) * temp_load
            left += rb
            right += ra

        if left + right > sumlr:
            sumlr = left + right
            dfSF.loc[f'{veh.name}', 'RL'], dfSF.loc[f'{veh.name}', 'RR'], dfSF.loc[
                f'{veh.name}', 'at'], dfSF.loc[f'{veh.name}', 'sum']= left, right, head_at, left + right

        if abs(left - right) > abs(difflr):
            difflr = left - right
            dfBM.loc[f'{veh.name}', 'RL'], dfBM.loc[f'{veh.name}', 'RR'], dfBM.loc[
                f'{veh.name}', 'at'], dfBM.loc[f'{veh.name}', 'sum'] = left, right, head_at, left + right
        head_at = round(head_at + step, 5)

In [55]:
total_frame = pd.concat([dfSF, dfBM], keys=['MaxSF', 'MaxBM'])
total_frame.to_excel('outputs/Live_Loads_Seismic.xlsx')
total_frame

RL        RR      at        sum
MaxSF Class A     240.2016  454.5144   69.57   694.7160
      Class 70RW  219.8760  911.7240   64.19  1131.6000
      Class 70RT  344.8160  349.3660   52.71   694.1820
MaxBM Class A     508.7832    0.0000   50.74   508.7832
      Class 70RW    0.0000  952.1040  109.08   952.1040
      Class 70RT  678.3700    0.0000   50.74   678.3700

**Horizontal forces due to water currents**

In [56]:
velocity = 1.77 # input

normal current - current parallel to pier (transverse to bridge alignment)

In [57]:
P_norm = f_watercurrent(velocity)[0]
F_norm = P_norm*pier_width*pier_height_hfl
# calculate moments here

current at 20 degrees to normal flow direction

In [58]:
P20_norm, P20_trans = f_watercurrent(velocity, 20)
F20_norm, F20_trans = P20_norm*pier_width*pier_height_hfl, P20_trans*pier_width*pier_height_hfl
# calculate moments here

**Longitudinal forces**

**In the case of a single lane or a two lane bridge** : twenty percent of the
first train load plus ten percent of the load of the succeeding trains or part
thereof, the train loads in one lane only being considered for the purpose of
this sub- clause. Where the entire first train is not on the full span, the
braking force shall be taken as equal to twenty percent of the loads actually
on the span or continuous unit of spans.

*longitudinal force acts along an line parallel to roadway and 1.2 m above it*

In [59]:
veh_for_long = ll_A.reduced(to=0.2)
for i in range(veh_no[0]-1):
    veh_for_long+=ll_A.reduced(to=0.1)
    
f_long = sum([i[1] for i in veh_for_long.loadpair if i[0]<span*2])

# calculate moment here

**Buoyancy**

To allow for full buoyancy, a reduction shall be made in the gross weight of
the member affected by reducing its density by the density of the displaced water.
*The density of water may be taken as 1.0 t/m$^{3}$*

In [60]:
cross_area = math.pi*0.8**2+6*1.6
volume = cross_area*pier_height_hfl
upthrust = 10*volume # kN
# calculate moment here

**Temperature and shrinkage**

In [61]:
temp_diff = 30 #degree celcius
alpha = 0.000012 # coeff of thermal exp
elongation_therm = round(alpha*temp_diff*span, 5)

shrink_strain = 0.0002
elongation_shrink = round(shrink_strain*50)